In [80]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import json
import numpy as np

from collections import Counter
from folium.plugins import MarkerCluster
from sklearn.cluster import DBSCAN

from utils import *
from FedRank import FedRank

In [2]:
# develop mode
%load_ext autoreload
%autoreload 2

In [3]:
# Moscow coordinates
MOW = [55.75222, 37.61556]

r = 0.02
r2 = r**2

In [48]:
# load, clean and group train data
train_df = clean_df('./data/train_set.csv')
train_gr = train_df.groupby('customer_id')

test_df = clean_df('./data/test_set.csv')
test_gr = test_df.groupby('customer_id')

/home/frodos/raifaizen/.env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/frodos/raifaizen/.env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### MCC
Подгружаем MCC коды, вычисляем их частотность в тренировочных данных.

In [67]:
# load mcc codes
mcc_codes = pd.read_csv('./data/mcc_codes.csv', index_col='mcc')
mcc_counts = train_df['mcc'].value_counts().to_frame(name = 'count')
mcc_codes = mcc_counts.merge(mcc_codes, left_index=True, right_index=True)
mcc_list = mcc_codes.query('count >10000').index.values

In [72]:
mcc_codes.loc[:,['count','edited_description']].head(20)

,count,edited_description
5411,391635,"Grocery Stores, Supermarkets"
6011,275228,Financial Institutions – Manual Cash Disbursem...
5814,128771,Fast Food Restaurants
5812,62407,Eating places and Restaurants
5499,44703,Misc. Food Stores – Convenience Stores and Spe...
5541,42296,Service Stations ( with or without ancillary s...
5912,41175,Drug Stores and Pharmacies
4111,24899,Local/Suburban Commuter Passenger Transportati...
5921,18353,"Package Stores – Beer, Wine, and Liquor"
5331,13042,Variety Stores


In [12]:
best_dbscan_params = {'eps': 0.036, 'min_samples': 2}

In [87]:
clusters = get_dbscan_clust(train_gr, best_dbscan_params, mcc_list=mcc_list)

In [91]:
train_df = train_df.merge(clusters, left_index=True, right_index=True, how='left')
train_df.clust_label.fillna(-2, inplace=True)
t_gr = train_df.groupby(['customer_id','clust_label'])

### Создаём допольнительные фичи
Проходимся по всем кластерам всех пользователей и считаем удельный вклад каждого MCC по количеству операций.

На выходе получаем таблицу, в которой колонки соответсвуют MCC

In [113]:
### Create features with relative contributions of each MCC code for each cluster of each customer
def get_mcc_features(gr):
    df = gr.mcc.aggregate('value_counts', **{'normalize':True}).unstack(level=-1).fillna(0)
    df.columns = ["mcc_%s" % (n1) for n1 in df.columns.values]
    return df

def get_clust_pos(gr):
    agg_dict = {
        'pos_address_lat':np.mean,
        'pos_address_lon': np.mean
        }
    return gr.aggregate(agg_dict)
    #newidx = []
    #for (n1,n2) in c.columns.ravel():
    #    newidx.append("%s_%s" % (n1,n2))
    #c.columns=newidx
    return c

In [119]:
def get_clust_features(gr):
    agg_dict = {
    'amount': [np.mean, np.max, np.min, 'count', 'std']
    }
    c = gr.aggregate(agg_dict)
    newidx = []
    for (n1,n2) in c.columns.ravel():
        newidx.append("%s_%s" % (n1,n2))
    c.columns=newidx
    return c

get_clust_features(t_gr)

amount_mean  amount_amax  \
customer_id                      clust_label                             
0001f322716470bf9bfc1708f06f00fc  0              3.309715     4.614833   
                                  1              2.553616     2.973381   
                                  2              3.255826     4.481342   
0007297d86e14bd68bd87b1dbdefe302 -1              2.351594     2.351594   
                                  0              2.491890     3.197908   
                                  1              2.792455     4.590551   
                                  2              2.820427     3.401110   
                                  3              2.367055     2.712781   
                                  4              2.951470     4.499151   
                                  5              2.488475     2.618730   
                                  6              2.159304     2.373256   
                                  7              2.910833     3.219585   
                                  8              2.843656     3.703698   
000b709c6c6fb1e8efcfd95e57c2a9de -1              2.178760     2.707712   
                                  0              2.454432     2.539817   
                                  1              2.701774     2.713293   
                                  2              2.813341     3.521626   
                                  3              3.050185     3.475695   
                                  4              2.567378     2.578458   
                                  5              1.665275     1.678664   
                                  6              3.459745     3.482396   
                                  7              3.034875     3.906433   
0027a7618d97cc9fbda55fac457eaeb7 -1              2.102438     2.102438   
                                  0              2.719135     3.224442   
                                  1              3.215212     3.359777   
                                  2              2.944809     3.375377   
002b9f6e118c54f1292e03d1a04d516e -1              3.194717     3.552046   
                                  0              2.353109     3.733430   
                                  1              2.757780     3.120491   
                                  2              3.103141     3.243828   
...                                                   ...          ...   
ff869ee855dc3f9b382c943eb43cc4ec  8              3.211753     3.255323   
                                  9              3.185266     3.284895   
                                  10             3.674559     4.562585   
ff92d5420f5fb92a37e1280d1fc9e5f4 -1              3.711420     3.711420   
                                  0              2.705312     3.104000   
                                  1              2.390978     2.498249   
                                  2              4.685161     4.687912   
ffaeae55d4dbf29058f04e7a6a764f02 -1              2.873541     2.873541   
                                  0              3.160077     3.224281   
                                  1              3.009963     3.534924   
                                  2              2.782018     3.070825   
ffc5289194413ec68c3f7adc8121d69b -1              2.680235     2.680235   
                                  0              2.173551     2.723632   
                                  1              2.772851     3.046552   
                                  2              2.514433     2.636701   
                                  3              3.343938     4.152719   
ffd097949a4a238296a7deadfb376cc0 -1              2.784659     2.784659   
                                  0              2.812380     3.445825   
                                  1              2.986624     3.170735   
                                  2              3.366384     3.366384   
                                  3              3.201811     3.201811   
ffdd5ec2a90e355cf40525eac1a6fd34 -1              2

In [120]:
t_gr.transform('sum')

ValueError: cannot reshape array of size 85 into shape (5,18)

In [ ]:
for k, v in t_gr.amount:
    print(k, v)

In [114]:
train_clust_pos = get_clust_pos(t_gr)
train_amount_features = get_clust_features(t_gr)
train_mcc_features = get_mcc_features(t_gr)

In [115]:
train_amount_features.merge(train_mcc_features, left_index=True, right_index=True)

amount_mean  amount_amax  \
customer_id                      clust_label                             
0001f322716470bf9bfc1708f06f00fc  0              3.309715     4.614833   
                                  1              2.553616     2.973381   
                                  2              3.255826     4.481342   
0007297d86e14bd68bd87b1dbdefe302 -1              2.351594     2.351594   
                                  0              2.491890     3.197908   
                                  1              2.792455     4.590551   
                                  2              2.820427     3.401110   
                                  3              2.367055     2.712781   
                                  4              2.951470     4.499151   
                                  5              2.488475     2.618730   
                                  6              2.159304     2.373256   
                                  7              2.910833     3.219585   
                                  8              2.843656     3.703698   
000b709c6c6fb1e8efcfd95e57c2a9de -1              2.178760     2.707712   
                                  0              2.454432     2.539817   
                                  1              2.701774     2.713293   
                                  2              2.813341     3.521626   
                                  3              3.050185     3.475695   
                                  4              2.567378     2.578458   
                                  5              1.665275     1.678664   
                                  6              3.459745     3.482396   
                                  7              3.034875     3.906433   
0027a7618d97cc9fbda55fac457eaeb7 -1              2.102438     2.102438   
                                  0              2.719135     3.224442   
                                  1              3.215212     3.359777   
                                  2              2.944809     3.375377   
002b9f6e118c54f1292e03d1a04d516e -1              3.194717     3.552046   
                                  0              2.353109     3.733430   
                                  1              2.757780     3.120491   
                                  2              3.103141     3.243828   
...                                                   ...          ...   
ff869ee855dc3f9b382c943eb43cc4ec  8              3.211753     3.255323   
                                  9              3.185266     3.284895   
                                  10             3.674559     4.562585   
ff92d5420f5fb92a37e1280d1fc9e5f4 -1              3.711420     3.711420   
                                  0              2.705312     3.104000   
                                  1              2.390978     2.498249   
                                  2              4.685161     4.687912   
ffaeae55d4dbf29058f04e7a6a764f02 -1              2.873541     2.873541   
                                  0              3.160077     3.224281   
                                  1              3.009963     3.534924   
                                  2              2.782018     3.070825   
ffc5289194413ec68c3f7adc8121d69b -1              2.680235     2.680235   
                                  0              2.173551     2.723632   
                                  1              2.772851     3.046552   
                                  2              2.514433     2.636701   
                                  3              3.343938     4.152719   
ffd097949a4a238296a7deadfb376cc0 -1              2.784659     2.784659   
                                  0              2.812380     3.445825   
                                  1              2.986624     3.170735   
                                  2              3.366384     3.366384   
                                  3              3.201811     3.201811   
ffdd5ec2a90e355cf40525eac1a6fd34 -1              2

In [116]:
dist = get_target_distances(train_df).fillna(999)
dist.head(10)

AttributeError: 'DataFrame' object has no attribute 'pos_address_lat_mean'

In [71]:
p = p.merge(dist, left_index=True, right_index=True)
p.head()

pos_address_lat_mean  \
customer_id                      clust_label                         
0001f322716470bf9bfc1708f06f00fc -2                      17.785491   
                                  0                      56.251347   
                                  1                      44.744977   
                                  2                      44.711199   
0007297d86e14bd68bd87b1dbdefe302 -2                      53.649054   

                                              pos_address_lon_mean  \
customer_id                      clust_label                         
0001f322716470bf9bfc1708f06f00fc -2                      15.020104   
                                  0                      43.446254   
                                  1                      37.725649   
                                  2                      37.846624   
0007297d86e14bd68bd87b1dbdefe302 -2                      35.916029   

                                              amount_mean  amount_amax  \
customer_id                      clust_label                             
0001f322716470bf9bfc1708f06f00fc -2              3.343322     4.614833   
                                  0              2.553616     2.973381   
                                  1              3.016776     3.460175   
                                  2              2.203973     2.394246   
0007297d86e14bd68bd87b1dbdefe302 -2              2.859040     4.696914   

                                              amount_amin  amount_count  \
customer_id                      clust_label                              
0001f322716470bf9bfc1708f06f00fc -2              1.982628            83   
                                  0              2.347082             7   
                                  1              2.407830             3   
                                  2              1.949756             7   
0007297d86e14bd68bd87b1dbdefe302 -2              0.706783           182   

                                              amount_std  home_add_lat_mean  \
customer_id                      clust_label                                  
0001f322716470bf9bfc1708f06f00fc -2             0.761167             44.708   
                                  0             0.233241             44.708   
                                  1             0.545355             44.708   
                                  2             0.160099             44.708   
0007297d86e14bd68bd87b1dbdefe302 -2             0.733852             55.799   

                                              home_add_lon_mean  \
customer_id                      clust_label                      
0001f322716470bf9bfc1708f06f00fc -2                      37.775   
                                  0                      37.775   
                                  1                      37.775   
                                  2                      37.775   
0007297d86e14bd68bd87b1dbdefe302 -2                      37.388   

                                              work_add_lat_mean  \
customer_id                      clust_label                      
0001f322716470bf9bfc1708f06f00fc -2                      44.735   
                                  0                      44.735   
                                  1                      44.735   
                                  2                      44.735   
0007297d86e14bd68bd87b1dbdefe302 -2                         NaN   

                                              work_add_lon_mean    dist_home  \
customer_id                      clust_label                                   
0001f322716470bf9bfc1708f06f00fc -2                      37.798  1242.606770   
                                  0                      37.798   165.411979   
                                  1                      37.798     0.003803   
                                  2                      37.798     0.005140   
0007297d86e14bd68bd87b1dbdefe302 -2          

In [72]:
p = p.reset_index().query('clust_label > -1')

In [73]:
ranker = FedRank()

In [80]:
features = ['dist_home', 'dist_work']
y = ranker.transform(p, features)
y.head(10)

,dist_home_rank,dist_work_rank
1,3,3
2,1,2
3,2,1
5,1,2
6,2,2
7,4,2
8,3,2
11,2,2
12,1,1
15,3,2


In [82]:
p.head()

,customer_id,clust_label,pos_address_lat_mean,pos_address_lon_mean,amount_mean,amount_amax,amount_amin,amount_count,amount_std,home_add_lat_mean,home_add_lon_mean,work_add_lat_mean,work_add_lon_mean,dist_home,dist_work
1,0001f322716470bf9bfc1708f06f00fc,0,56.251347,43.446254,2.553616,2.973381,2.347082,7,0.233241,44.708,37.775,44.735,37.798,165.411979,164.529019
2,0001f322716470bf9bfc1708f06f00fc,1,44.744977,37.725649,3.016776,3.460175,2.407830,3,0.545355,44.708,37.775,44.735,37.798,0.003803,0.005334
3,0001f322716470bf9bfc1708f06f00fc,2,44.711199,37.846624,2.203973,2.394246,1.949756,7,0.160099,44.708,37.775,44.735,37.798,0.005140,0.002931
5,0007297d86e14bd68bd87b1dbdefe302,0,55.800632,37.400692,2.834341,3.596210,0.816582,40,0.560534,55.799,37.388,NaN,NaN,0.000164,999.000000
6,0007297d86e14bd68bd87b1dbdefe302,1,55.822604,37.524894,2.820427,3.401110,2.359619,3,0.530993,55.799,37.388,NaN,NaN,0.019297,999.000000


In [83]:
features = ['amount_mean', 'amount_count', 'amount_std']
X = ranker.transform(p, features)
X.head()

,amount_mean_rank,amount_count_rank,amount_std_rank
1,2,2,2
2,3,1,3
3,1,2,1
5,4,4,4
6,3,1,3


In [84]:
y.head()

,dist_home_rank,dist_work_rank
1,3,3
2,1,2
3,2,1
5,1,2
6,2,2


### Обучение и тестирование

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Подбор гиперпараметров для DBSCAN

In [ ]:
eps_list = np.logspace(-2, -1, 5)
min_samples_list = [5, 10, 15, 20, 25, 30]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

In [ ]:
eps_list = np.logspace(-3, -1, 10)
min_samples_list = [1, 2, 3, 4, 5]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

Наилучшие результаты получены при eps = 0.036, min_samples = 1

In [ ]:
train_pr = get_dbscan_score(train_gr, best_dbscan_params)

In [ ]:
train_pr.head()

In [ ]:
train_pr.clust_amount.hist(bins = 100)
plt.xlim([0, 20])

Теперь оценим реальный скор, который я могу выбить

In [ ]:
0.5* train_pr.query('dist <= @r2').shape[0]/train_pr.shape[0]

На борде скор 0.206375

In [ ]:
test_pr = get_dbscan_score(test_gr, best_dbscan_params, calc_dist= False)
test_pr.head()

In [ ]:
submit = test_pr.loc[:,['home_post_lat', 'home_post_lon', 'best_post_lat',  'best_post_lon']]
submit.head()

In [ ]:
submit.columns = ['best_post_lat', 'best_post_lon',  'home_post_lat', 'home_post_lon', 'dist', 'clust_size']

In [ ]:
submit.to_csv('1.csv')